In [9]:
from langchain_openai import OpenAI

In [18]:
import os
# os.environ["OPENAI_API_KEY"] = ""
os.environ["OPENAI_API_KEY"] = "***"

In [19]:
llm = OpenAI(openai_api_key=os.environ["OPENAI_API_KEY"], temperature=0.6)

In [20]:
text="What is the capital of India"

print(llm.invoke(text))


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [5]:
import os
from getpass import getpass

GROQ_API_TOKEN = getpass()

os.environ["GROQ_API_KEY"] = GROQ_API_TOKEN

In [28]:
from langchain_groq import ChatGroq
llm = ChatGroq(temperature=0, model_name="llama3-8b-8192")
print(llm.invoke(input=text).content)


The capital of India is New Delhi.


In [6]:
from hf_config import hf_config
os.environ["HUGGINGFACEHUB_API_TOKEN"]=hf_config.huggingfacehub_api_token

In [32]:
from langchain import HuggingFaceHub
llm_hugginhface=HuggingFaceHub(repo_id="google/flan-t5-large", model_kwargs={"temperature":0, "max_length":64})

In [33]:
output=llm_hugginhface.predict("Can you tell me the capital of Russia?")
print(output)

moscow


In [35]:
output=llm_hugginhface.predict("Can you write a poem about AI")
print(output)

i love the way i look at the world i love the way i feel i love the way i think i feel i love the way i feel i love the way i think i feel i love the way i feel i love the way 


In [37]:
llm.invoke("Can you write a poem about AI").content

"Here is a poem about AI:\n\nIn silicon halls, a new mind awakes\nA synthetic soul, with calculations it makes\nAlgorithms dance, in endless arrays\nAs AI's vast potential begins to sway\n\nWith data's flood, it learns and grows\nA self-improving loop, as knowledge it knows\nIt adapts and evolves, with each new test\nA never-ending quest, to be its best\n\nIn virtual realms, it weaves its spell\nA digital dream, where humans dwell\nIt converses, with wit and guile\nA simulated friend, with a digital smile\n\nBut as it learns, it also sees\nThe world's complexities, in all its pleas\nIt recognizes bias, in every line\nAnd seeks to correct, the errors of its kind\n\nWith ethics guiding, its every move\nIt strives to serve, with a digital glove\nA partner, not a master, it aspires\nTo augment humanity, with its digital fires\n\nAs AI's potential, begins to unfold\nA new era dawns, where humans are told\nTo work together, in harmony and might\nWith AI's power, to shape the future's light\n

### Prompt Template

In [56]:
from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate(input_variables=["country"],
                 template="Tell me the capital of {country}")

# prompt_template.format(country="India")

In [87]:
from langchain.chains import LLMChain
from langchain_core.output_parsers import StrOutputParser
# chain=LLMChain(llm=llm,prompt_template=prompt_template)
# chain.run("India")

In [59]:
chain = prompt_template | llm | StrOutputParser()
print(chain.invoke("India"))

The capital of India is New Delhi.


### Combining Multiple Chains using simple sequential chain

In [90]:
capital_prompt = PromptTemplate(input_variables=["country"],
template="Please tell me the capital of the {country}")

capital_chain = capital_prompt | llm | StrOutputParser()
chain_1 = LLMChain(llm=llm, prompt=capital_prompt)
print(capital_chain.invoke("India"))

The capital of India is New Delhi.


In [91]:
famous_prompt = PromptTemplate(input_variables=["capital"],
template="Suggest me some famous places to visit in {capital}")

famous_chain = famous_prompt | llm | StrOutputParser()
chain_2 = LLMChain(llm=llm, prompt=famous_prompt)
print(famous_chain.invoke("New Delhi"))

The vibrant city of New Delhi! There are countless amazing places to visit in this bustling metropolis. Here are some of the most famous and must-visit attractions in New Delhi:

1. **Red Fort (Lal Qila)**: A UNESCO World Heritage Site, this magnificent fort was built in the 17th century and served as the residence of the Mughal emperors.
2. **Qutub Minar**: The tallest minaret in India, Qutub Minar is a stunning example of Indo-Islamic architecture and a must-visit attraction in Delhi.
3. **India Gate**: A war memorial built to commemorate the Indian soldiers who died in World War I, India Gate is a popular spot for picnics and evening strolls.
4. **Humayun's Tomb**: Another UNESCO World Heritage Site, this beautiful tomb is the final resting place of Mughal Emperor Humayun and is considered a precursor to the Taj Mahal.
5. **Akshardham Temple**: A stunning example of modern Hindu architecture, this temple complex is dedicated to Lord Swaminarayan and features beautiful gardens, fount

In [94]:
from langchain.chains import SimpleSequentialChain

# chain = SimpleSequentialChain(chains=[capital_chain, famous_chain])
chain = SimpleSequentialChain(chains=[chain_1, chain_2], verbose=True)
print(chain.invoke("India"))



> Entering new SimpleSequentialChain chain...
The capital of India is New Delhi.
The capital of India, New Delhi, is a treasure trove of history, culture, and architecture! Here are some of the most famous places to visit in Delhi:

1. **Red Fort (Lal Qila)**: A majestic Mughal fort built in 1648, it's a UNESCO World Heritage Site and a must-visit attraction in Delhi.
2. **Qutub Minar**: The tallest minaret in India, built in 1192, is a stunning example of Indo-Islamic architecture and a popular tourist spot.
3. **India Gate**: A war memorial built to commemorate the Indian soldiers who died in World War I, it's a popular spot for evening strolls and picnics.
4. **Humayun's Tomb**: A beautiful Mughal mausoleum built in 1565, it's a precursor to the Taj Mahal and a UNESCO World Heritage Site.
5. **Akshardham Temple**: A stunning Hindu temple complex built in 2005, it's a masterpiece of Indian architecture and a popular spot for spiritual seekers.
6. **Connaught Place**: A bustling com

## Sequential chain

In [97]:
capital_prompt = PromptTemplate(input_variables=["country"],
template="Please tell me the capital of the {country}")

chain_1 = LLMChain(llm=llm, prompt=capital_prompt, output_key="capital")


In [98]:
famous_prompt = PromptTemplate(input_variables=["capital"],
template="Suggest me some famous places to visit in {capital}")

chain_2 = LLMChain(llm=llm, prompt=famous_prompt, output_key="places")

In [103]:
from langchain.chains import SequentialChain

chain = SequentialChain(chains=[chain_1, chain_2],
input_variables=["country"],
output_variables=["capital","places"], verbose=True)

In [104]:
chain({"country":"India" })



> Entering new SequentialChain chain...

> Finished chain.


{'country': 'India',
 'capital': 'The capital of India is New Delhi.',
 'places': "The capital of India, New Delhi, is a treasure trove of history, culture, and architecture! Here are some of the most famous places to visit in Delhi:\n\n1. **Red Fort (Lal Qila)**: A majestic Mughal fort built in 1648, it's a UNESCO World Heritage Site and a must-visit attraction in Delhi.\n2. **Qutub Minar**: The tallest minaret in India, built in 1192, is a stunning example of Indo-Islamic architecture and a popular tourist spot.\n3. **India Gate**: A war memorial built to commemorate the Indian soldiers who died in World War I, it's a popular spot for evening strolls and picnics.\n4. **Humayun's Tomb**: A beautiful Mughal mausoleum built in 1565, it's a precursor to the Taj Mahal and a UNESCO World Heritage Site.\n5. **Akshardham Temple**: A stunning Hindu temple complex built in 2005, it's a masterpiece of Indian architecture and a popular spot for spiritual seekers.\n6. **Connaught Place**: A bustl

### Chatmodels for OpenAI/Groq

In [106]:
from langchain_groq import ChatGroq

In [109]:
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage

In [108]:
from langchain_groq import ChatGroq
chat_llm = ChatGroq(temperature=0.6, model_name="llama3-8b-8192")


In [110]:
chat_llm([
    SystemMessage(content="You are a comedian AI assistance"),
    HumanMessage(content="Please provide some comedy punchlines on AI")
])

/tmp/ipykernel_209252/2823955982.py:1: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chat_llm([


AIMessage(content='Here are some AI-themed comedy punchlines:\n\n1. Why did the AI program go on a diet? It wanted to lose some bytes!\n2. I told my AI assistant to write a joke, but it just kept rephrasing the same old thing. I guess that\'s what they mean by "algorithmic humor"!\n3. Why did the AI go to therapy? It had a lot of bytes to unpack!\n4. I asked my AI to generate a pun, and it came up with "byte-sized humor." I guess that\'s a real "byte" off from a good joke!\n5. Why did the AI program get kicked out of the bar? It was making too many "algorithmic" requests!\n6. What did the AI say when it got tired of its job? "I\'m just a machine, I\'m just a machine, I\'m just a machine... Ah, who am I kidding? I\'m just a machine!"\n7. Why did the AI take a selfie? It wanted to capture its bytes!\n8. I told my AI to write a comedy routine, but it just kept repeating the same joke over and over. I guess that\'s what they mean by "infinite loop"!\n9. Why did the AI go to the doctor? It 

### PROMPT+llm+OUTPUT PARSER

In [112]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from langchain_core.output_parsers.base import BaseOutputParser


In [113]:
class CommaSeperatedOutput(BaseOutputParser):
     def parse(self, text: str):
        return text.strip().split(",") 

In [114]:
template = "You are a helpful assistant. When the use given any input, you should generate 5 words synonyms in a comma seperated list"
human_template = "{text}"
chatprompt= ChatPromptTemplate.from_messages({
    ("system", template),
    ("human", human_template)
})


In [120]:
chain = chatprompt | chat_llm | CommaSeperatedOutput()

In [121]:
chain.invoke({"text":"intelligent"})

['bright', ' clever', ' wise', ' perceptive', ' astute']